In [88]:
from pydoc import classname
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.firefox.service import Service as FirefoxService
from selenium.webdriver.ie.service import Service as IEService
from selenium.webdriver.common.keys import Keys
from ipywidgets import widgets
from IPython.display import FileLink
from datetime import date
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os


In [91]:
# Get what we need from Forward Real Estate - a parcel # and what exact site to search
print("This is for searching New Castle County \n\n")
#parcel_number = input('Enter your parcel number: ')
parcel_label = widgets.Label('Parcel Number')
display(parcel_label)
parcel_input = widgets.Text()
display(parcel_input)
button = widgets.Button(description="Get Infomation")
display(button)
todays_date = date.today()


def get_parcel_info(b):
    parcel_number = parcel_input.value
    service = ChromeService(executable_path=ChromeDriverManager().install())

    website_search_url = 'http://www3.nccde.org/parcel/search/'

    print("PARCEL NUMBER", parcel_number)
    # Navigate to the site
    driver = webdriver.Chrome(service.path)
    driver.implicitly_wait(3)
    driver.get(website_search_url)

    # for now we'll use an implicit wait for our waiting strategy --- WILL CHANGE cc selenium site
    #Get the parcel number box
    parcel_search_box = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1__TextBoxParcelNumber"]')
    parcel_search_box.send_keys(parcel_number)
    search_button = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1__ButtonSearch"]')
    search_button.click()
    details = driver.find_element_by_xpath('//*[@id="ctl00_ctl00_ContentPlaceHolder1_ContentPlaceHolder1__GridViewResults_ctl02__LinkButtonDetails"]')
    details.click()
    time.sleep(5)
    detail_url = driver.current_url
    driver.quit()

    #parsing HTML
    page = requests.get(detail_url)
    page_html = BeautifulSoup(page.text, 'html.parser')
    residence_characteristics = page_html.find("div", class_="residence level4")
    table = residence_characteristics.find("table", class_="form")
    all_rows = table.find_all('tr')

    ## Creating the df
    parcel_dictionary = {'Parcel #': parcel_number}
    house_data = []
    for row in all_rows:
        columns = row.find_all('td')
        columns = [elem.text.strip() for elem in columns]
        house_data.append([elem for elem in columns])


    features = []
    data = []
    for sublist in house_data:
        for i in range(len(sublist)):
            if i % 2 == 0:
                elem = sublist[i].replace(':', '')
                features.append(elem)
            else:
                data.append(sublist[i])

    for i in range(len(features)):
        parcel_dictionary[features[i]] = data[i]

    del parcel_dictionary['']
    parcel_data_df = pd.DataFrame([parcel_dictionary])


    # export to excel
    parcel_data_df.to_excel(f'./{todays_date}-parcel_sheet_export.xlsx', index = False)
    #display(FileLink(f'./{todays_date}-parcel_sheet_export.xlsx'))
    print('Your excel sheet is done! Thanks!')
    display(FileLink('README.md'))

button2 = widgets.Button(description="Get Excel Sheet")
output = widgets.Output()
display(button2, output)
def get_excel_sheet_link(b):
    with output:
        display(FileLink(f'./{todays_date}-parcel_sheet_export.xlsx'))

button.on_click(get_parcel_info)
button2.on_click(get_excel_sheet_link)


This is for searching New Castle County 




Label(value='Parcel Number')

Text(value='')

Button(description='Get Infomation', style=ButtonStyle())

Button(description='Get Excel Sheet', style=ButtonStyle())

Output()